In [59]:
!pip install bltk
!pip install bnlp
!pip install bnlp_toolkit
!pip install git+https://github.com/banglakit/lemmatizer.git#egg=banglakit-lemmatizer
!pip install bangla-stemmer
!pip install globe

  Obtaining dependency information for bnlp_toolkit from https://files.pythonhosted.org/packages/af/99/5d688fc2e5f6494d6c0a4e6bcdc8192266869caeef3b41dbc3dc8358616d/bnlp_toolkit-4.0.0-py3-none-any.whl.metadata
  Using cached bnlp_toolkit-4.0.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached sentencepiece-0.1.99-cp311-cp311-win_amd64.whl (977 kB)
  Using cached sklearn_crfsuite-0.3.6-py2.py3-none-any.whl (12 kB)
  Using cached ftfy-6.1.1-py3-none-any.whl (53 kB)
  Using cached emoji-1.7.0-py3-none-any.whl
  Using cached python-crfsuite-0.9.9.tar.gz (440 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using cached bnlp_toolkit-4.0.0-py3-none-any.whl (22 kB)
  Running setup.py clean for python-crfsuite
Failed to build python-crfsuite


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [12 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-311
  creating build\lib.win-amd64-cpython-311\pycrfsuite
  copying pycrfsuite\_dumpparser.py -> build\lib.win-amd64-cpython-311\pycrfsuite
  copying pycrfsuite\_logparser.py -> build\lib.win-amd64-cpython-311\pycrfsuite
  copying pycrfsuite\__init__.py -> build\lib.win-amd64-cpython-311\pycrfsuite
  running build_ext
  building 'pycrfsuite._pycrfsuite' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for python-crfsuite
ERROR: Could not build wheels for python-crfsuite, which is requi

  Cloning https://github.com/banglakit/lemmatizer.git to c:\users\taufiq\appdata\local\temp\pip-install-q_xb4ck9\banglakit-lemmatizer_89efcf825b6c45daa35a4d81b31bd275


  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git version
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [60]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk

CPU times: total: 0 ns
Wall time: 997 µs


In [61]:
raw_train_data = pd.read_excel('train.xlsx', index_col = 0) 
raw_val_data = pd.read_excel('val.xlsx', index_col = 0)
raw_test_data = pd.read_excel('test.xlsx', index_col = 0)

In [62]:
raw_train_data.head()

,text,classes,labels
0,মেডিকেলে এসে ঠিক ফ্যামিলির ফটো ফ্রেম এর মতো মন...,sadness,1
1,অনেক দিন পর অন্য রকম অনুভূতি হচ্ছে কেননা আমার...,sadness,1
2,আগে যতোটা বেশি পছন্দ করতাম দুশ্চরিত্রা মিথিলা ...,disgust,3
3,মাঝে মাঝে মনে হয় তোমাকে চাওয়াটা অনেক ভুল হয়ত ভ...,sadness,1
4,এখানে কাওকে কবর দিয়ো না তাহলে তোমাদের পরিবারের...,fear,5


In [63]:
raw_val_data['text'][0]

'তোরা সবাই বিটিভির পর্দায় ফিরে আয়🙂তোদের জন্য মুভি নারেবাম্পার ফলনই তোদের বিনোদনের মূল মাধ্যম 🙄পোলাপান কতটা সিরিয়াস মাইরিভাবা যায় সামান্য মুভির সামান্য রিভিউ বিনিময়ে কী সাংঘাতিক গালাগাল🙄ম্যাঁ গো'

In [64]:
#The Bengali Characters

from bltk.langtools.banglachars import (vowels,
                                        vowel_signs,
                                        consonants,
                                        digits,
                                        operators,
                                        punctuations,
                                        others)
print(f'Vowels: {vowels}')
print(f'Vowel signs: {vowel_signs}')
print(f'Consonants: {consonants}')
print(f'Digits: {digits}')
print(f'Operators: {operators}')
print(f'Punctuation marks: {punctuations}')
print(f'Others: {others}')

Vowels: ['অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'ঌ', 'এ', 'ঐ', 'ও', 'ঔ']
Vowel signs: ['া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ৄ', 'ে', 'ৈ', 'ো', 'ৌ']
Consonants: ['ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', 'ড়', 'ঢ়', 'য়', 'ৎ', 'ং', 'ঃ', 'ঁ']
Digits: ['০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯']
Operators: ['=', '+', '-', '*', '/', '%', '<', '>', '×', '÷']
Punctuation marks: ['।', ',', ';', ':', '?', '!', "'", '.', '"', '-', '[', ']', '{', '}', '(', ')', '–', '—', '―', '~']
Others: ['৳', '৺', '্', 'ঀ', 'ঽ', '#', '$']


In [65]:
from bltk.langtools import Tokenizer, remove_stopwords
from bangla_stemmer.stemmer import stemmer

In [66]:
def preprocess_bangla_text(text):
    
    #Tokenize the text
    tokenizer = Tokenizer()
    words = tokenizer.word_tokenizer(text)
    
    #Remove Stopword
    filtered_words = remove_stopwords(words)
    
    #Stem Words:
    stmer = stemmer.BanglaStemmer()
    stemmed_words = [stmer.stem(word) for word in filtered_words]
    stemmed_text = " ".join(stemmed_words)
    
    #remove HTML tag from the text
    html_tag = re.compile(r'<.*?>')
    html_clean_text = re.sub(html_tag,'',stemmed_text)
    
    #remove url
    url_clean = re.compile(r"https://\S+|www\.\S+")
    url_clean_text = re.sub(url_clean,'',html_clean_text)
    
    #remove emojies
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U0001FAE0-\U0001FAFF"  # Symbols for Legacy Computing
                           u"\U0001FAC0-\U0001FAFF"  # Additional emoticons
                           u"\U0001F004-\U0001F0CF"  # Miscellaneous Symbols and Pictographs
                           u"\U0001F170-\U0001F251"  # Enclosed Alphanumeric Supplement
                           "]+", flags=re.UNICODE)
    emoji_clean_text = re.sub(emoji_pattern,'',url_clean_text)
    
    preprocessed_text = emoji_clean_text
    return preprocessed_text
    
    
    
    
    
    
    

In [67]:
text = "তোরা সবাই বিটিভির পর্দায় ফিরে আয়🙂তোদের জন্য মুভি নারেবাম্পার ফলনই তোদের বিনোদনের মূল মাধ্যম 🙄পোলাপান কতটা সিরিয়াস মাইরিভাবা যায় সামান্য মুভির সামান্য রিভিউ বিনিময়ে কী সাংঘাতিক গালাগাল🙄ম্যাঁ গো"

In [68]:
print(preprocess_bangla_text(text))

applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
তোরা সবা বিটিভির পর্দায় ফিরে আয়তো মুভি নারেবাম্পার ফলন তোদের বিনোদন মূল মাধ্যম পোলাপান কত সিরিয়াস মাইরিভাবা সামান্য মুভির সামান্য রিভিউ বিনিময় সাংঘাতিক গালাগালম্যাঁ গো


In [69]:
raw_train_data['clean_text'] = raw_train_data['text'].apply(preprocess_bangla_text)

applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fo

applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
app

applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied second rules..
applied second rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied second rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
a

applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied

applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied second rules.

...

applied third rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied third rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
ap

applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
ap

applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..

applied fourth rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied second rules..
applied second rules..
applied second rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules

applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied third rules..
applied second rules..
applied fourth rules..
applied third rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied third rules..
applied second rules..
appli

In [70]:
raw_train_data.head()

,text,classes,labels,clean_text
0,মেডিকেলে এসে ঠিক ফ্যামিলির ফটো ফ্রেম এর মতো মন...,sadness,1,মেডিকেল এসে ফ্যামিলির ফটো ফ্রেম মত মনে হত নিজে...
1,অনেক দিন পর অন্য রকম অনুভূতি হচ্ছে কেননা আমার...,sadness,1,অনেক দিন পর অন্য রকম অনুভূতি কেননা সদ্য পাশ কর...
2,আগে যতোটা বেশি পছন্দ করতাম দুশ্চরিত্রা মিথিলা ...,disgust,3,যতো বেশি পছন্দ করতাম দুশ্চরিত্ মিথিলা মুখার্জি...
3,মাঝে মাঝে মনে হয় তোমাকে চাওয়াটা অনেক ভুল হয়ত ভ...,sadness,1,মনে তোমাকে চাওয়া অনেক ভুল হয়ত ভুল তবে ভুল মাশু...
4,এখানে কাওকে কবর দিয়ো না তাহলে তোমাদের পরিবারের...,fear,5,কাওকে কবর দিয়ো তোমা পরিবার আর কেউ বেঁচে থাকবে ...


## extract feature:

In [71]:
train_texts = raw_train_data.clean_text.tolist()

In [72]:
train_labels = raw_train_data.labels.tolist()

In [73]:
for i in range(5):
    print(train_texts[i])

মেডিকেল এসে ফ্যামিলির ফটো ফ্রেম মত মনে হত নিজেকেখাপছাড়া লাগতোদম হয় আসতোঅন্য সামনে অপমানিত হত পদে পদে
অনেক দিন পর অন্য রকম অনুভূতি কেননা সদ্য পাশ কর শিক্ষার্থীর মত এখন আমি বেক বেক পরিচয় দিয়ে আম্ম কাছ আজ পঞ্চাশ টাকা ডাকাতি কর
যতো বেশি পছন্দ করতাম দুশ্চরিত্ মিথিলা মুখার্জি এখন তেমনি সবচ বেশি ঘৃণা করি ওই পতিতা সদ্য হিন্দু হওয়া মিথিলা মুখার্জি
মনে তোমাকে চাওয়া অনেক ভুল হয়ত ভুল তবে ভুল মাশুল এইভাবেই দিত হত জানি তবে যা জানি মানি
কাওকে কবর দিয়ো তোমা পরিবার আর কেউ বেঁচে থাকবে ও তোমা কাউ বাঁচ দিবে চলে যাও এখান নিজে অভিশাপ আহ্বান করো


In [74]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

CPU times: total: 0 ns
Wall time: 0 ns


# Embedding and model learning with tf-idf vectorizer

In [75]:
lr_model = LogisticRegressionCV(class_weight='balanced', max_iter=400)
dt_model = DecisionTreeClassifier(class_weight='balanced')
rf_model = RandomForestClassifier(class_weight='balanced')
svm_model = SVC(class_weight='balanced')
knn_model = KNeighborsClassifier()
model_names = ['Logistic Regression','Decision Tree','Random Forest', 'SVM', 'KNN']
ml_models = [lr_model,dt_model,rf_model, svm_model, knn_model]

In [76]:
%%time
def model_train(model, train_data, train_labels):
  pipe = Pipeline([
        ('FE', TfidfVectorizer(ngram_range=(1,3), use_idf = True, tokenizer=lambda x: x.split(), max_features=20000)),  
        ('Transformer', TfidfTransformer()),  
        ('M', model)
    ])
  pipe.fit(train_data, train_labels)
  return pipe

trained_ml_models = []
for i in ml_models:
    tt = model_train(i,train_texts, train_labels)
    trained_ml_models.append(tt)

C:\Users\Taufiq\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\Taufiq\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Taufiq\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/st

CPU times: total: 6min
Wall time: 4min 1s


# Test the models for Test and Validation data set

In [77]:
raw_test_data['clean_text'] = raw_test_data['text'].apply(preprocess_bangla_text)
raw_val_data['clean_text'] = raw_val_data['text'].apply(preprocess_bangla_text)

applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
app

applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied s

applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rule

applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied third rules..
applied third rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied

applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rul

...

applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied second rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applie

applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applie

applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied s

applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied third rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
appl

applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
app

In [78]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

## Test Dataset

In [79]:
test_texts = raw_test_data.clean_text.tolist()
test_labels = raw_test_data.labels.tolist()

In [90]:
##test data
pr_list = []
re_list = []
f1_list = []
accuracy_list = []


def print_results(md, x_test, y_test, name):
#     print("\n\nClassifier: ", name)
    pred_y = md.predict(x_test)
    true, pred = y_test, pred_y
#     print(confusion_matrix(true,pred))
#     print(classification_report(true,pred))
#     print("\nPrecison : ",precision_score(pred,true, average = 'weighted'))
#     print("\nRecall : ",recall_score(pred,true,  average = 'weighted'))
#     print("\nF1 : ",f1_score(pred,true,  average = 'weighted'))

    pr_list.append((round(precision_score(y_test, pred_y, average='weighted'),4)*100)) 
    re_list.append((round(recall_score(y_test, pred_y, average='weighted'),4)*100))
    f1_list.append((round(f1_score(y_test, pred_y, average='weighted'),4)*100))
    accuracy_list.append(accuracy_score(y_test, pred_y, normalize=True)*100)

for i in range(len(trained_ml_models)):
    md = trained_ml_models[i]
    name = model_names[i]
    print_results(md, test_texts, test_labels, name)

performance_matrix = pd.DataFrame({'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list, 'Accuracy':accuracy_list},
                                  index =model_names)
performance_matrix

,Precision,Recall,F1 Score,Accuracy
Logistic Regression,61.89,60.00,60.37,60.00
Decision Tree,47.19,46.08,46.49,46.08
Random Forest,58.61,57.92,57.06,57.92
SVM,63.43,62.72,62.44,62.72
KNN,42.75,41.60,40.89,41.60


## Validation Dataset

In [92]:
val_texts = raw_val_data.clean_text.tolist()
val_labels = raw_val_data.labels.tolist()

In [94]:
##val data
pr_list = []
re_list = []
f1_list = []
accuracy_list = []


def print_results(md, x_test, y_test, name):
#     print("\n\nClassifier: ", name)
    pred_y = md.predict(x_test)
    true, pred = y_test, pred_y
#     print(confusion_matrix(true,pred))
#     print(classification_report(true,pred))
#     print("\nPrecison : ",precision_score(pred,true, average = 'weighted'))
#     print("\nRecall : ",recall_score(pred,true,  average = 'weighted'))
#     print("\nF1 : ",f1_score(pred,true,  average = 'weighted'))

    pr_list.append((round(precision_score(y_test, pred_y, average='weighted'),4)*100)) 
    re_list.append((round(recall_score(y_test, pred_y, average='weighted'),4)*100))
    f1_list.append((round(f1_score(y_test, pred_y, average='weighted'),4)*100))
    accuracy_list.append(accuracy_score(y_test, pred_y, normalize=True)*100)

for i in range(len(trained_ml_models)):
    md = trained_ml_models[i]
    name = model_names[i]
    print_results(md, val_texts,val_labels, name)

performance_matrix = pd.DataFrame({'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list, 'Accuracy':accuracy_list},
                                  index =model_names)
performance_matrix

,Precision,Recall,F1 Score,Accuracy
Logistic Regression,63.46,62.50,62.81,62.500000
Decision Tree,48.90,48.56,48.68,48.557692
Random Forest,59.95,59.46,58.91,59.455128
SVM,65.45,65.06,64.59,65.064103
KNN,47.63,46.79,45.88,46.794872


# ML model with with FastText Embadding

In [95]:
%%time
import fasttext.util
import fasttext
import numpy as np

# Load pre-trained FastText embeddings for Bengali
model_fasttext = fasttext.load_model('cc.bn.300.bin')  # Load the FastText model

CPU times: total: 18.5 s
Wall time: 1min 59s


In [96]:
%%time
text_embeddings = []

# Iterate through each text
for text in train_texts:
    tokens = text.split()  # Tokenize the text (split by space, you can use a more advanced tokenizer)
    token_embeddings = []  # To store embeddings for individual tokens in the text
    for token in tokens:
        token_embedding = model.get_word_vector(token)  # Generate FastText embedding for the token
        token_embeddings.append(token_embedding)
    # Calculate the mean of token embeddings to represent the entire text
    if token_embeddings:
        text_embedding = np.mean(token_embeddings, axis=0)
        text_embeddings.append(text_embedding)
    else:
        # Handle cases where there are no tokens (empty text)
        text_embeddings.append(np.zeros(300))

CPU times: total: 1min 27s
Wall time: 8min 21s


In [97]:
%%time
def model_train(model, train_data, train_labels):
    pipe_fasttext = Pipeline([
        ('M', model)
        ])
    pipe_fasttext.fit(train_data, train_labels)
    return pipe_fasttext

trained_ml_models = []
for i in ml_models:
    tt = model_train(i,text_embeddings, train_labels)
    trained_ml_models.append(tt)

C:\Users\Taufiq\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Taufiq\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

CPU times: total: 1min 56s
Wall time: 1min 21s


# Test Dataset

In [98]:
test_texts = raw_test_data.clean_text.tolist()
test_labels = raw_test_data.labels.tolist()

In [99]:
text_embeddings = []

# Iterate through each text
for text in test_texts:
    tokens = text.split()  # Tokenize the text (split by space, you can use a more advanced tokenizer)
    token_embeddings = []  # To store embeddings for individual tokens in the text
    for token in tokens:
        token_embedding = model.get_word_vector(token)  # Generate FastText embedding for the token
        token_embeddings.append(token_embedding)
    # Calculate the mean of token embeddings to represent the entire text
    if token_embeddings:
        text_embedding = np.mean(token_embeddings, axis=0)
        text_embeddings.append(text_embedding)
    else:
        # Handle cases where there are no tokens (empty text)
        text_embeddings.append(np.zeros(300))

In [100]:
##test data
pr_list = []
re_list = []
f1_list = []
accuracy_list = []


def print_results(md, x_test, y_test, name):
#     print("\n\nClassifier: ", name)
    pred_y = md.predict(x_test)
    true, pred = y_test, pred_y
#     print(confusion_matrix(true,pred))
#     print(classification_report(true,pred))
#     print("\nPrecison : ",precision_score(pred,true, average = 'weighted'))
#     print("\nRecall : ",recall_score(pred,true,  average = 'weighted'))
#     print("\nF1 : ",f1_score(pred,true,  average = 'weighted'))

    pr_list.append((round(precision_score(y_test, pred_y, average='weighted'),4)*100)) 
    re_list.append((round(recall_score(y_test, pred_y, average='weighted'),4)*100))
    f1_list.append((round(f1_score(y_test, pred_y, average='weighted'),4)*100))
    accuracy_list.append(accuracy_score(y_test, pred_y, normalize=True)*100)

for i in range(len(trained_ml_models)):
    md = trained_ml_models[i]
    name = model_names[i]
    print_results(md, text_embeddings, test_labels, name)

performance_matrix = pd.DataFrame({'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list, 'Accuracy':accuracy_list},
                                  index =model_names)
performance_matrix

,Precision,Recall,F1 Score,Accuracy
Logistic Regression,55.91,53.76,53.98,53.76
Decision Tree,36.49,36.64,36.45,36.64
Random Forest,53.91,51.68,48.22,51.68
SVM,56.97,54.56,54.91,54.56
KNN,45.95,45.12,44.40,45.12


# validation dataset

In [101]:
val_texts = raw_val_data.clean_text.tolist()
val_labels = raw_val_data.labels.tolist()

In [102]:
text_embeddings = []

# Iterate through each text
for text in val_texts:
    tokens = text.split()  # Tokenize the text (split by space, you can use a more advanced tokenizer)
    token_embeddings = []  # To store embeddings for individual tokens in the text
    for token in tokens:
        token_embedding = model.get_word_vector(token)  # Generate FastText embedding for the token
        token_embeddings.append(token_embedding)
    # Calculate the mean of token embeddings to represent the entire text
    if token_embeddings:
        text_embedding = np.mean(token_embeddings, axis=0)
        text_embeddings.append(text_embedding)
    else:
        # Handle cases where there are no tokens (empty text)
        text_embeddings.append(np.zeros(300))

In [103]:
##val data
pr_list = []
re_list = []
f1_list = []
accuracy_list = []


def print_results(md, x_test, y_test, name):
#     print("\n\nClassifier: ", name)
    pred_y = md.predict(x_test)
    true, pred = y_test, pred_y
#     print(confusion_matrix(true,pred))
#     print(classification_report(true,pred))
#     print("\nPrecison : ",precision_score(pred,true, average = 'weighted'))
#     print("\nRecall : ",recall_score(pred,true,  average = 'weighted'))
#     print("\nF1 : ",f1_score(pred,true,  average = 'weighted'))

    pr_list.append((round(precision_score(y_test, pred_y, average='weighted'),4)*100)) 
    re_list.append((round(recall_score(y_test, pred_y, average='weighted'),4)*100))
    f1_list.append((round(f1_score(y_test, pred_y, average='weighted'),4)*100))
    accuracy_list.append(accuracy_score(y_test, pred_y, normalize=True)*100)

for i in range(len(trained_ml_models)):
    md = trained_ml_models[i]
    name = model_names[i]
    print_results(md, text_embeddings,val_labels, name)

performance_matrix = pd.DataFrame({'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list, 'Accuracy':accuracy_list},
                                  index =model_names)
performance_matrix

,Precision,Recall,F1 Score,Accuracy
Logistic Regression,59.71,57.21,57.69,57.211538
Decision Tree,38.02,38.78,38.24,38.782051
Random Forest,54.79,54.01,51.07,54.006410
SVM,61.74,59.46,59.91,59.455128
KNN,44.15,43.59,43.11,43.589744


# ML model with text embadding with GLOBE dataset

In [106]:
glove_model = {}

with open('bn_glove.39M.300d.txt', 'r', encoding='utf-8') as file:
    for line in file:
        parts = line.strip().split()
        word = parts[0]
        embedding = np.array(parts[1:], dtype=np.float32)
        glove_model[word] = embedding

In [107]:
%%time
text_embeddings = []

for text in train_texts:
    tokens = text.split()
    token_embeddings = []
    for token in tokens:
        if token in glove_model:
            token_embedding = glove_model[token]
            token_embeddings.append(token_embedding)
    if token_embeddings:
        text_embedding = np.mean(token_embeddings, axis=0)
        text_embeddings.append(text_embedding)
    else:
        text_embeddings.append(np.zeros(glove_model.vector_size))

CPU times: total: 266 ms
Wall time: 288 ms


In [108]:
%%time
def model_train(model, train_data, train_labels):
    pipe_fasttext = Pipeline([
        ('M', model)
        ])
    pipe_fasttext.fit(train_data, train_labels)
    return pipe_fasttext

trained_ml_models = []
for i in ml_models:
    tt = model_train(i,text_embeddings, train_labels)
    trained_ml_models.append(tt)

C:\Users\Taufiq\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Taufiq\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

CPU times: total: 1min 50s
Wall time: 1min 23s


# Test dataset

In [109]:
test_texts = raw_test_data.clean_text.tolist()
test_labels = raw_test_data.labels.tolist()

In [110]:
%%time
text_embeddings = []

for text in test_texts:
    tokens = text.split()
    token_embeddings = []
    for token in tokens:
        if token in glove_model:
            token_embedding = glove_model[token]
            token_embeddings.append(token_embedding)
    if token_embeddings:
        text_embedding = np.mean(token_embeddings, axis=0)
        text_embeddings.append(text_embedding)
    else:
        text_embeddings.append(np.zeros(glove_model.vector_size))

CPU times: total: 46.9 ms
Wall time: 50.1 ms


In [111]:
##test data
pr_list = []
re_list = []
f1_list = []
accuracy_list = []


def print_results(md, x_test, y_test, name):
#     print("\n\nClassifier: ", name)
    pred_y = md.predict(x_test)
    true, pred = y_test, pred_y
#     print(confusion_matrix(true,pred))
#     print(classification_report(true,pred))
#     print("\nPrecison : ",precision_score(pred,true, average = 'weighted'))
#     print("\nRecall : ",recall_score(pred,true,  average = 'weighted'))
#     print("\nF1 : ",f1_score(pred,true,  average = 'weighted'))

    pr_list.append((round(precision_score(y_test, pred_y, average='weighted'),4)*100)) 
    re_list.append((round(recall_score(y_test, pred_y, average='weighted'),4)*100))
    f1_list.append((round(f1_score(y_test, pred_y, average='weighted'),4)*100))
    accuracy_list.append(accuracy_score(y_test, pred_y, normalize=True)*100)

for i in range(len(trained_ml_models)):
    md = trained_ml_models[i]
    name = model_names[i]
    print_results(md, text_embeddings, test_labels, name)

performance_matrix = pd.DataFrame({'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list, 'Accuracy':accuracy_list},
                                  index =model_names)
performance_matrix

,Precision,Recall,F1 Score,Accuracy
Logistic Regression,51.77,50.40,50.18,50.40
Decision Tree,35.06,34.72,34.70,34.72
Random Forest,50.13,48.64,44.98,48.64
SVM,49.98,48.80,48.37,48.80
KNN,39.90,39.84,38.56,39.84


# validation dataset

In [115]:
val_texts = raw_val_data.clean_text.tolist()
val_labels = raw_val_data.labels.tolist()

In [116]:
%%time
text_embeddings = []

for text in val_texts:
    tokens = text.split()
    token_embeddings = []
    for token in tokens:
        if token in glove_model:
            token_embedding = glove_model[token]
            token_embeddings.append(token_embedding)
    if token_embeddings:
        text_embedding = np.mean(token_embeddings, axis=0)
        text_embeddings.append(text_embedding)
    else:
        text_embeddings.append(np.zeros(glove_model.vector_size))

CPU times: total: 31.2 ms
Wall time: 52.2 ms


In [117]:
##val data
pr_list = []
re_list = []
f1_list = []
accuracy_list = []


def print_results(md, x_test, y_test, name):
#     print("\n\nClassifier: ", name)
    pred_y = md.predict(x_test)
    true, pred = y_test, pred_y
#     print(confusion_matrix(true,pred))
#     print(classification_report(true,pred))
#     print("\nPrecison : ",precision_score(pred,true, average = 'weighted'))
#     print("\nRecall : ",recall_score(pred,true,  average = 'weighted'))
#     print("\nF1 : ",f1_score(pred,true,  average = 'weighted'))

    pr_list.append((round(precision_score(y_test, pred_y, average='weighted'),4)*100)) 
    re_list.append((round(recall_score(y_test, pred_y, average='weighted'),4)*100))
    f1_list.append((round(f1_score(y_test, pred_y, average='weighted'),4)*100))
    accuracy_list.append(accuracy_score(y_test, pred_y, normalize=True)*100)

for i in range(len(trained_ml_models)):
    md = trained_ml_models[i]
    name = model_names[i]
    print_results(md, text_embeddings,val_labels, name)

performance_matrix = pd.DataFrame({'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list, 'Accuracy':accuracy_list},
                                  index =model_names)
performance_matrix

,Precision,Recall,F1 Score,Accuracy
Logistic Regression,54.74,53.37,53.57,53.365385
Decision Tree,37.00,38.46,37.43,38.461538
Random Forest,52.53,51.12,48.17,51.121795
SVM,56.66,55.29,55.60,55.288462
KNN,42.38,41.67,40.20,41.666667
